In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
import anndata
import loompy as lp

os.chdir('/workspace/Benchmarking')
from metrics import *

## Run pyscenic and compute AUC matrix for regulon enrichment (Lymphoma)

In [ ]:
# Inference based on true RNA data
run_pyscenic('/workspace/Benchmarking/data_scbutterfly/lymphoma_RNA_hv.h5ad', '/workspace/Benchmarking/data_all', 'babel')
auc_mtx_true = compute_auc(input_path ='/workspace/Benchmarking/data_all/babeldata_scenic.loom', data_dir ='/workspace/Benchmarking/data_all', output_file_name ='babel_auc_true.loom', loom_file_name = '')

# scButterfly prediction
auc_mtx_pred_sb = compute_auc('/workspace/scButterfly/data/lymphoma/predicted/pred_RNA_lymphoma2.h5ad', '/workspace/Benchmarking/data_all', 'lymphoma_scenic_pred.loom', 'auc_pred.loom')

# Babel prediction
auc_mtx_pred_babel = compute_auc('/workspace/babel/mymodel/atac_rna_test_preds_pp.h5ad', '/workspace/Benchmarking/data_all', 'babeldata_scenic_pred.loom', 'auc_babel_pred.loom')

# Polar prediction
auc_mtx_pred_polar = compute_auc(input_path ='/workspace/Benchmarking/data_all/polar_pred.h5ad', data_dir ='/workspace/Benchmarking/data_all', output_file_name ='polar_auc_pred.loom', loom_file_name = 'ploar_pred.loom')

In [ ]:
# Bootstrpping AUC calculation for normalization
bootstrap_results = bootstrap_auc('/workspace/Benchmarking/data_all/babeldata_scenic.loom', '/workspace/Benchmarking/data_all', n_iterations=20, seed=999)
print('Babel bootstrap results:', bootstrap_results)

Import prerun inference and AUC predictions from Archived files

In [ ]:
# lf = lp.connect('data_all/babel_auc_true.loom', mode='r+', validate=False )
# auc_mtx_true = pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)
# lf.close()

# lf = lp.connect('data_all/auc_babel_pred.loom', mode='r+', validate=False )
# auc_mtx_pred_babel = pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)
# lf.close()

# lf = lp.connect('data_all/auc_pred.loom', mode='r+', validate=False )
# auc_mtx_pred_sb = pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)
# lf.close()

# lf = lp.connect('data_all/polar_auc_pred.loom', mode='r+', validate=False )
# auc_mtx_pred_polar = pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)
# lf.close()

In [ ]:
babel_grcs = compute_grcs(auc_mtx_true, auc_mtx_pred_babel)
sb_grcs = compute_grcs(auc_mtx_true, auc_mtx_pred_sb)
polar_grcs = compute_grcs(auc_mtx_true, auc_mtx_pred_polar)

print(babel_grcs, sb_grcs, polar_grcs)

Plot results in a Violon plot

In [ ]:
violon_df = pd.DataFrame({
    'BABEL': list(babel_grcs[1].values()),
    'scButterfly': list(sb_grcs[1].values()),
    'Polarbear': list(polar_grcs[1].values())
})

paired_colors = [
    "#ef973f",  # light orange/T cells
    "#87e175",  # light green/ mono
    "#ba8ad3",  # light purple/Tumour B cells
]

fig, ax = plt.subplots(figsize=(5, 5))
sns.violinplot(data=violon_df, palette=paired_colors, linewidth=2, ax=ax, fontsize=16, width=0.6, scale='area')

for spine in ax.spines.values():
    spine.set_visible(False)

ax.spines['left'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.set_ylabel('Correlation', fontsize=16)
ax.tick_params(axis='x', labelsize=16)

plt.tight_layout()
plt.savefig('/workspace/Benchmarking/figures/violin_grcs_lymphoma.svg', format='svg', bbox_inches='tight')
plt.show()

### Differential Analysis of Regulons (Lymphmome dataset)

In [ ]:
babel_true = anndata.AnnData(auc_mtx_true, dtype='float64')
babel_pred = anndata.AnnData(auc_mtx_pred_babel, dtype='float64')
babel_true_test = babel_true[babel_pred.obs_names].copy()
babel_pred.obs['Cell Types'] = pred_RNA_babel.obs['Cell Types']
babel_true_test.obs['Cell Types'] = pred_RNA_babel.obs['Cell Types']

sb_pred = anndata.AnnData(auc_mtx_pred_sb, dtype='float64')
sb_pred.obs['Cell Types'] = pred_RNA_sb.obs['Cell Types']

polar_pred = anndata.AnnData(auc_mtx_polar, dtype='float64')
polar_pred.obs['Cell Types'] = pred_RNA_polar.obs['Cell Types']

In [ ]:
babel_true_test.obs['group'] = babel_true_test.obs['Cell Types'].map(
    lambda x: 'B' if x in ['B'] else ('LYM' if x in ['Tumor B', 'Tumor B cycling'] else None)
)
data_true = babel_true_test[babel_true_test.obs['group'].isin(['B','LYM'])].copy()

babel_pred.obs['group'] = babel_pred.obs['Cell Types'].map(
    lambda x: 'B' if x in ['B'] else ('LYM' if x in ['Tumor B', 'Tumor B cycling'] else None)
)
data_babel_pred = babel_pred[babel_pred.obs['group'].isin(['B','LYM'])].copy()

sb_pred.obs['group'] = sb_pred.obs['Cell Types'].map(
    lambda x: 'B' if x in ['B'] else ('LYM' if x in ['Tumor B', 'Tumor B cycling'] else None)
)
data_sb_pred = sb_pred[sb_pred.obs['group'].isin(['B','LYM'])].copy()

polar_pred.obs['group'] = polar_pred.obs['Cell Types'].map(
    lambda x: 'B' if x in ['B'] else ('LYM' if x in ['Tumor B', 'Tumor B cycling'] else None)
)
data_polar_pred = polar_pred[polar_pred.obs['group'].isin(['B','LYM'])].copy()

sc.tl.rank_genes_groups(
    data_true,
    groupby='group',
    groups=['LYM'],
    reference='B',
    method='wilcoxon',
    pts=True
)

sc.tl.rank_genes_groups(
    data_babel_pred,
    groupby='group',
    groups=['LYM'],
    reference='B',
    method='wilcoxon',
    pts=True
)

sc.tl.rank_genes_groups(
    data_sb_pred,
    groupby='group',
    groups=['LYM'],
    reference='B',
    method='wilcoxon',
    pts=True
)

sc.tl.rank_genes_groups(
    data_polar_pred,
    groupby='group',
    groups=['LYM'],
    reference='B',
    method='wilcoxon',
    pts=True
)

de_true = sc.get.rank_genes_groups_df(data_true, group='LYM')
de_true['neg_log10_padj'] = -np.log10(de_true['pvals_adj'].replace(0, np.nan))

de_babel = sc.get.rank_genes_groups_df(data_babel_pred, group='LYM')
de_babel['neg_log10_padj'] = -np.log10(de_babel['pvals_adj'].replace(0, np.nan))

de_sb = sc.get.rank_genes_groups_df(data_sb_pred, group='LYM')
de_sb['neg_log10_padj'] = -np.log10(de_sb['pvals_adj'].replace(0, np.nan))

de_polar = sc.get.rank_genes_groups_df(data_polar_pred, group='LYM')
de_polar['neg_log10_padj'] = -np.log10(de_polar['pvals_adj'].replace(0, np.nan))

df1 = de_true
df2 = de_babel
df3 = de_sb
df4 = de_polar

Resulting Volcano plots

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Example dataframes df1, df2, df3, df4 (replace these with your actual dataframes)
axs = axes.flatten()
# Plot for df1
df1['color'] = 'grey'
df1.loc[(df1['logfoldchanges'] > 1) & (df1['pvals_adj'] < 0.001), 'color'] = '#fcad45'
df1.loc[(df1['logfoldchanges'] < -1) & (df1['pvals_adj'] < 0.001), 'color'] = '#60b1dd'

axes[0, 0].scatter(df1['logfoldchanges'], df1['neg_log10_padj'], s=20, alpha=1, c=df1['color'], edgecolor='none')
axes[0, 0].axhline(-np.log10(0.001), color='green', linestyle='--')
axes[0, 0].axvline(1, color='green', linestyle='dotted')
axes[0, 0].axvline(-1, color='green', linestyle='dotted')
axes[0, 0].set_xlim(-2, 4)
axes[0, 0].set_title('True RNA data', fontsize=16)
axes[0, 0].set_xlabel('log\u2082(FC)', fontsize=14)
axes[0, 0].set_ylabel('-log\u2081\u2080(p-value)', fontsize=14)

for _, row in df1.loc[((df1['logfoldchanges'] > 1) & (df1['pvals_adj'] < 0.001)) | ((df1['logfoldchanges'] < -1) & (df1['pvals_adj'] < 0.001)), :].iterrows():
    axes[0,0].text(row['logfoldchanges'], row['neg_log10_padj'],
             row['names'], fontsize=10, ha='center', va='bottom', color='black')

# Plot for df2
df2['color'] = 'grey'
df2.loc[(df2['logfoldchanges'] > 1) & (df2['pvals_adj'] < 0.001), 'color'] = '#fcad45'
df2.loc[(df2['logfoldchanges'] < -1) & (df2['pvals_adj'] < 0.001), 'color'] = '#60b1dd'

axes[0, 1].scatter(df2['logfoldchanges'], df2['neg_log10_padj'], s=20, alpha=0.7, c=df2['color'], edgecolor='none')
axes[0, 1].axhline(-np.log10(0.001), color='green', linestyle='--')
axes[0, 1].axvline(1, color='green', linestyle='dotted')
axes[0, 1].axvline(-1, color='green', linestyle='dotted')
axes[0, 1].set_xlim(-5, 4)
axes[0, 1].set_title('Predicted BABEL', fontsize=16)
axes[0, 1].set_xlabel('log\u2082(FC)', fontsize=14)
axes[0, 1].set_ylabel('-log\u2081\u2080(p-value)', fontsize=14)

for _, row in df2.loc[((df2['logfoldchanges'] > 1) & (df2['pvals_adj'] < 0.001)) | ((df2['logfoldchanges'] < -1) & (df2['pvals_adj'] < 0.001)), :].iterrows():
    axes[0,1].text(row['logfoldchanges'], row['neg_log10_padj'],
             row['names'], fontsize=10, ha='center', va='bottom', color='black')

# Plot for df3
df3['color'] = 'grey'
df3.loc[(df3['logfoldchanges'] > 1) & (df3['pvals_adj'] < 0.001), 'color'] = '#fcad45'
df3.loc[(df3['logfoldchanges'] < -1) & (df3['pvals_adj'] < 0.001), 'color'] = '#60b1dd'

axes[1, 0].scatter(df3['logfoldchanges'], df3['neg_log10_padj'], s=20, alpha=0.7, c=df3['color'], edgecolor='none')
axes[1, 0].axhline(-np.log10(0.001), color='green', linestyle='--')
axes[1, 0].axvline(1, color='green', linestyle='dotted')
axes[1, 0].axvline(-1, color='green', linestyle='dotted')
axes[1, 0].set_xlim(-3, 3)
axes[1, 0].set_title('Predicted scButterfly', fontsize=16)
axes[1, 0].set_xlabel('log\u2082(FC)', fontsize=14)
axes[1, 0].set_ylabel('-log\u2081\u2080(p-value)', fontsize=14)

for _, row in df3.loc[((df3['logfoldchanges'] > 1) & (df3['pvals_adj'] < 0.001)) | ((df3['logfoldchanges'] < -1) & (df3['pvals_adj'] < 0.001)), :].iterrows():
    axes[1,0].text(row['logfoldchanges'], row['neg_log10_padj'],
             row['names'], fontsize=10, ha='center', va='bottom', color='black')

# Plot for df4
df4['color'] = 'grey'
df4.loc[(df4['logfoldchanges'] > 1) & (df4['pvals_adj'] < 0.001), 'color'] = '#fcad45'
df4.loc[(df4['logfoldchanges'] < -1) & (df4['pvals_adj'] < 0.001), 'color'] = '#60b1dd'

axes[1, 1].scatter(df4['logfoldchanges'], df4['neg_log10_padj'], s=20, alpha=0.7, c=df4['color'], edgecolor='none')
axes[1, 1].axhline(-np.log10(0.001), color='green', linestyle='--')
axes[1, 1].axvline(1, color='green', linestyle='dotted')
axes[1, 1].axvline(-1, color='green', linestyle='dotted')
axes[1, 1].set_xlim(-5, 3)
axes[1, 1].set_title('Predicted Polarbear', fontsize=16)
axes[1, 1].set_xlabel('log\u2082(FC)', fontsize=14)
axes[1, 1].set_ylabel('-log\u2081\u2080(p-value)', fontsize=14)

for _, row in df4.loc[((df4['logfoldchanges'] > 1) & (df4['pvals_adj'] < 0.001)) | ((df4['logfoldchanges'] < -1) & (df4['pvals_adj'] < 0.001)), :].iterrows():
    axes[1,1].text(row['logfoldchanges'], row['neg_log10_padj'],
             row['names'], fontsize=10, ha='center', va='bottom', color='black')


for axis in axs:
    axis.spines['top'].set_visible(False)
    axis.spines['right'].set_visible(False)

# Adjust layout to avoid overlap
plt.tight_layout()
plt.subplots_adjust(hspace=0.2) 
plt.savefig('/workspace/Benchmarking/figures/grn_deg_lymphoma_.svg', format='svg', bbox_inches='tight')
plt.show()

## GRCS analysis for BMMC dataset

In [ ]:
# true RNA data
run_pyscenic('/workspace/scButterfly/data/bmmc/ds_RNA_bmmc_hv.h5ad', '/workspace/Benchmarking/data_bmmc', model='scButterfly')
auc_mtx_bmmc_true = compute_auc(input_path ='/workspace/Benchmarking/data_bmmc/scButterflydata_scenic.loom', data_dir = '/workspace/Benchmarking/data_bmmc', output_file_name = 'bmmc_auc_true.loom', loom_file_name = '')

# scButterfly prediction
auc_mtx_bmmc_sb = compute_auc('/workspace/Benchmarking/data_bmmc/bmmc_sb.h5ad', '/workspace/Benchmarking/data_bmmc', 'Polardata_scenic_pred.loom', 'auc_sb_pred.loom')

# Polarbear prediction
auc_mtx_bmmc_polar = compute_auc('/workspace/Benchmarking/data_bmmc/bmmc_polar.h5ad', '/workspace/Benchmarking/data_bmmc', 'scButterflydata_scenic_pred.loom', 'auc_polar_pred.loom')

In [ ]:
# lf = lp.connect('data_bmmc/bmmc_auc_true.loom', mode='r+', validate=False )
# auc_mtx_bmmc_true = pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)
# lf.close()

# lf = lp.connect('data_all/auc_sb_pred.loom', mode='r+', validate=False )
# auc_mtx_bmmc_sb = pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)
# lf.close()

# lf = lp.connect('data_scbutterfly/auc_polar_pred.loom', mode='r+', validate=False )
# auc_mtx_bmmc_polar = pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)
# lf.close()

In [ ]:
bootstrap_results_bmmc = bootstrap_auc('/workspace/Benchmarking/data_bmmc/scButterflydata_scenic.loom', '/workspace/Benchmarking/data_bmmc', n_iterations=20, seed=999)
print('Babel bootstrap results:', bootstrap_results_bmmc)

In [ ]:
polar_grcs_bmmc = compute_grcs(auc_mtx_bmmc_true, auc_mtx_bmmc_polar)
sb_grcs_bmmc = compute_grcs(auc_mtx_bmmc_true, auc_mtx_bmmc_sb)

print(polar_grcs_bmmc, sb_grcs_bmmc)

In [ ]:
violon_df_bmmc = pd.DataFrame({
    'scButterfly': list(sb_grcs_bmmc[1].values()),
    'Polarbear': list(polar_grcs_bmmc[1].values())
})

paired_colors = [ 
    "#ba8ad3",  # light purple/Tumour B cells
    '#ff7f00',  # orange/Mixed
]

fig, ax = plt.subplots(figsize=(5, 5))
sns.violinplot(data=violon_df_bmmc, palette=paired_colors, linewidth=2, ax=ax, fontsize=16, width=0.5, scale='area')

# Remove the box around the plot (hide the spines)
for spine in ax.spines.values():
    spine.set_visible(False)

# Optionally, keep only the left and bottom spines visible
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.tick_params(axis='x', labelsize=16)

# Show the plot
plt.tight_layout()
plt.savefig('/workspace/Benchmarking/figures/violin_grcs_bmmc2.svg', format='svg', bbox_inches='tight')
plt.show()